In [20]:

import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import svm
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error as mae
from sklearn import preprocessing

import imblearn
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE




In [16]:

# import packages for hyperparameters tuning
#from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

dataset = pd.read_csv('../data/training_set/TRAIN_ME_carriers.csv')
data = dataset.copy()
data['mkt_carrier'].unique()


le = preprocessing.LabelEncoder()
data['mkt_carrier'] = le.fit_transform(data['mkt_carrier'])

data = data.drop(['Unnamed: 0'],axis=1)
data['flight_status'] = (data['arr_delay'] > 15).replace([True,False],[1,0])
data.drop(['arr_delay'],axis=1,inplace=True)
data['speed']=data['distance']/data['crs_elapsed_time']

# sample1 = data.loc[data['flight_status'] == 1]
# sample2 = data.loc[data['flight_status'] == 0]
# sample_2 = sample2.sample(n=757903)
# sample = sample1.append(sample_2)

sample = data.sample(n=2000)


X = sample.drop(['flight_status'], axis=1)
y = sample['flight_status']


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [21]:
model2 = xgb.XGBClassifier()
model2.load_model("xgb_classifier.json")

XGBoostError: [09:57:55] /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/common/io.cc:102: Opening xgb_classifier.json failed: No such file or directory
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000001a1c1230fe dmlc::LogMessageFatal::~LogMessageFatal() + 110
  [bt] (1) 2   libxgboost.dylib                    0x0000001a1c146e8c xgboost::common::LoadSequentialFile(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, bool) + 828
  [bt] (2) 3   libxgboost.dylib                    0x0000001a1c11b946 XGBoosterLoadModel + 614
  [bt] (3) 4   libffi.6.dylib                      0x00000001053ca934 ffi_call_unix64 + 76
  [bt] (4) 5   ???                                 0x00007ffeeb3b9d80 0x0 + 140732844973440



In [12]:
#### SMOTE sampling

sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [14]:
y_train_res.shape

(2576,)

In [ ]:

##### XGBoost
model_imb = xgb.XGBClassifier(alpha=1,
                             learning_rate = 0.1,
                             max_depth = 6,
                             min_child_weight = 9,
                             n_estimators = 100,
                             objective = 'binary:logistic',
                             scale_pos_weight = 0.75,
                             subsample = 0.7)
                                   
model_imb.fit(X, y)

print("Accuracy on training set: {:.2f}".format(model_imb.score(X_train, y_train) * 100))
print("Accuracy on validation set: {:.2f}".format(model_imb.score(X_test, y_test) * 100))

xgb_predict = model_imb.predict(X_test)
xgb_predict

print(confusion_matrix(y_test,xgb_predict))
print(classification_report(y_test,xgb_predict))